In [1]:
import requests
import sqlite3
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import os

In [2]:
#time.time()

In [3]:
if 'log.txt' in os.listdir():
    with open('log.txt', 'r') as f:
        if f.read() == '':
            f.close()
            with open('log.txt','w+') as f:
                f.write(f'{0},{time.time()}')
else:
    with open('log.txt', 'w') as f:
        f.write(f'{0},{time.time()}')

In [18]:
def internet_connection():
    try:
        response = requests.get("https://www.google.com", timeout=5)
        return True
    except requests.ConnectionError:
        return False

In [4]:
def get_base_url(inp="https://api.divar.ir/v8/web-search/30/plot-old"):
    inp2 = input('Enter your url :')
    if inp2 == '':
        return inp
    else:
        return inp2


def get_base_url2(inp=None):
    if inp:
        return inp
    else:
        return "https://api.divar.ir/v8/web-search/30/plot-old"


def get_last_post_data(inp=None):
    if inp:
        return inp
    else:
        return int(str(time.time()).replace('.', ''))

In [5]:
base_url = get_base_url2()
last_post_date = get_last_post_data()

In [6]:
def get_tokens(inp_url, inp_last_post_date):
    tokens = []
    k = 0
    l_p_date = inp_last_post_date
    while True:
        js_v = {"page": k,
                "json_schema": {"category": {"value": "plot-old"}, "query": "زمین", "sort": {"value": "sort_date"}},
                "last-post-date": l_p_date}
        res_v = requests.post(url=base_url, json=js_v)
        if res_v.status_code == 200:
            #print(k, ' done', end='')
            data_v = res_v.json()
            l_p_date = data_v['last_post_date']
            for j in data_v["web_widgets"]["post_list"]:
                try:
                    tokens.append(j['data']['token'])
                except:
                    continue
            print(f'{k}:done', end=' ')
        else:
            print('end')
            break
        k += 1
        time.sleep(0.5)

    return tokens

In [7]:
def second_get_tokens(inp_url, inp_date, end_date):
    tokens = []
    k = 0
    l_p_date = inp_date
    while True:
        js_v = {"page": k,
                "json_schema": {"category": {"value": "plot-old"}, "query": "زمین", "sort": {"value": "sort_date"}},
                "last-post-date": l_p_date}
        res_v = requests.post(url=base_url, json=js_v)
        if res_v.json()['last_post_date'] > end_date and res_v.status_code == 200:
            #print(k, ' done', end='')
            data_v = res_v.json()
            l_p_date = data_v['last_post_date']
            for j in data_v["web_widgets"]["post_list"]:
                try:
                    tokens.append(j['data']['token'])
                except:
                    continue
            print(f'{k}:done', end=' ')
        else:
            print('end')
            break
        k += 1
        time.sleep(0.5)
    return tokens

In [8]:
def adad_farsi_to_english(inp: str):
    dc = {'۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4', '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'}
    out = ''
    for i in inp:
        if i in dc.keys():
            out += dc[i]
        else:
            out += ''
    if out.isdecimal():
        return int(out)
    else:
        return None

In [9]:
def get_data_by_tokens(inp_tokens):
    df_dict = {'addres': [], 'area': [], 'price': []}
    k = 1
    for i in inp_tokens:
        print(k, end=' ')
        url_v = "https://api.divar.ir/v8/posts-v2/web/" + i
        res_v = requests.get(url_v)
        if res_v.status_code == 200:
            data_v = res_v.json()
            try:
                df_dict['addres'].append(data_v['seo']['title'])
                for j in range(len(data_v['sections'])):
                    if data_v['sections'][j]['section_name'] == 'LIST_DATA':
                        df_dict['area'].append(
                            adad_farsi_to_english(data_v['sections'][j]['widgets'][0]['data']['value']))
                        df_dict['price'].append(adad_farsi_to_english(data_v[
                                                                          'sections'][j]['widgets'][1]['data'][
                                                                          'value']))
                        #df_dict['unit_price'].append(adad_farsi_to_english(data_v[
                        #'sections'][j]['widgets'][2]['data']['value']))
            except:
                continue
        time.sleep(0.5)
        k += 1
    return pd.DataFrame(df_dict)

In [10]:
#a = pd.DataFrame({'addres': [], 'area': [], 'price': []})
#a.loc[len(a.index) + 1] = ['a', 200, 1000]
#a

In [11]:
#a.loc[len(a.index) + 1] = ['b', 2, 2000]
#a

In [12]:
#a.loc[len(a.index) + 1] = ['c', 3, 3500]
#a

In [13]:
def first_get_data_by_tokens(inp_tokens):
    df_func = pd.DataFrame({'addres': [], 'area': [], 'price': []})
    addres=''
    area=0
    price=0
    for i in tqdm(inp_tokens):
        #df_item = []
        try:
            url_v = "https://api.divar.ir/v8/posts-v2/web/" + i
            res_v = requests.get(url_v)
            if res_v.status_code == 200:
                try:
                    data_v = res_v.json()
                    #print(k, end=' ')
                    addres=data_v['seo']['title']
                    for j in range(len(data_v['sections'])):
                        if data_v['sections'][j]['section_name'] == 'LIST_DATA':
                            area=adad_farsi_to_english(data_v['sections'][j]['widgets'][0]['data']['value'])
                            price=adad_farsi_to_english(data_v['sections'][j]['widgets'][1]['data']['value'])
                            #print([addres,area,price])
                            df_func.loc[len(df_func.index) + 1] = [addres,area,price]
                            break

                except:
                    continue
        except:
            continue

    return df_func

In [14]:
def second_get_data_by_tokens(inp_tokens):
    df_func = pd.read_excel('out_data.xlsx')
    for i in tqdm(inp_tokens):
        try:
            url_v = "https://api.divar.ir/v8/posts-v2/web/" + i
            res_v = requests.get(url_v)
            if res_v.status_code == 200:
                data_v = res_v.json()
                try:
                    adres = data_v['seo']['title']
                    for j in range(len(data_v['sections'])):
                        if data_v['sections'][j]['section_name'] == 'LIST_DATA':
                            area = adad_farsi_to_english(data_v['sections'][j]['widgets'][0]['data']['value'])
                            price = adad_farsi_to_english(data_v[
                                                              'sections'][j]['widgets'][1]['data'][
                                                              'value'])
                            df_func.loc[len(df_func.index) + 1] = [adres, area, price]
                            break
                except:
                    continue
        except:
            continue
    return df_func

In [15]:
#first_url='https://api.divar.ir/v8/web-search/kashan/buy-old-house?goods-business-type=all&q=%D8%B2%D9%85%DB%8C%D9%86'
#first_param='goods-business-type=all&q=%D8%B2%D9%85%DB%8C%D9%86'
#requests.get(first_url,params=first_param).status_code
#a=requests.get(first_url,params=first_param).json()

In [16]:
#a['first_post_date']

In [17]:
if 'log.txt' in os.listdir() and internet_connection():
    first_url = 'https://api.divar.ir/v8/web-search/kashan/buy-old-house?goods-business-type=all&q=%D8%B2%D9%85%DB%8C%D9%86'
    first_param = 'goods-business-type=all&q=%D8%B2%D9%85%DB%8C%D9%86'
    f = open('log.txt', 'r')
    msg = f.read().split(',')
    f.close()
    if msg[0] == '0':
        f = open('log.txt', 'w+')
        a = requests.get(url=first_url, params=first_param).json()['first_post_date']
        f.write(f'{1},{time.time()},{a}')
        f.close()
        tokens_ = get_tokens(base_url, last_post_date)
        df = first_get_data_by_tokens(tokens_)
        df.to_excel('out_data.xlsx')
    else:
        f = open('log.txt', 'w+')
        #end_point_last_post_date = int(msg[2].replace('.', ''))
        a = requests.get(url=first_url, params=first_param).json()['first_post_date']
        f.write(f'{int(msg[0]) + 1},{time.time()},{a}')
        f.close()
        end_date = int(msg[2])
        tokens_ = second_get_tokens(base_url, last_post_date, end_date)
        df = second_get_data_by_tokens(tokens_)
        df.to_excel('out_data.xlsx')

end


0it [00:00, ?it/s]
